# Langchain是什么
Langchain可以被看成一个AI应用的开发框架，它把AI应用中会用到的组件分成以下几类：
- 模型 Models
- 提示词 Prompts
- 索引 Indexes
- Memory
- Chains 
- Agents

Langchain的价值在于，它提供了一层抽象，使得应用开发中的逻辑层和实现层可以解耦，比如模型部分可以使用GPT-3.5、GPT-4.0、GLM平滑替换。这层抽象也使扩展程序变得更简单，可以很轻松的引入新的功能。

## 模型 Models
目前Langchain支持三种模型：
- LLMs(Large Language Models)：接受文本输入，返回文本作为输出
- Chat Models: 本质上还是LLMs，只是输入输出是有一定结构的聊天信息
- Text Embedding Models: 接受文本输入，返回浮点数列表作为输出

## 提示词 Prompts

## 索引 Indexes

## Memory

## Chains

## Agents